<a href="https://colab.research.google.com/github/sunjana2199/ML-Climate-Final-Project-Template/blob/main/src/Twitter_Data2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install snscrape

In [12]:
# pip install --upgrade pip'

In [2]:
!pip install git+https://github.com/JustAnotherArchivist/snscrape.git

  Cloning https://github.com/JustAnotherArchivist/snscrape.git to /tmp/pip-req-build-hu7x46bm
  Running command git clone -q https://github.com/JustAnotherArchivist/snscrape.git /tmp/pip-req-build-hu7x46bm
ERROR: Package 'snscrape' requires a different Python: 3.7.13 not in '~=3.8'


In [3]:
import pandas as pd
import time
from datetime import datetime, timedelta
import snscrape.modules.twitter as sntwitter

In [4]:
top_anomaly_dates = pd.read_csv('/content/top_anomaly_dates.csv')
bottom_anomaly_dates = pd.read_csv('/content/bottom_anomaly_dates.csv')

In [5]:
top_start_dates = top_anomaly_dates['Date'].to_list()
top_end_dates = []
for i in top_anomaly_dates.Date:
  k = int(i[8:]) + 1
  end_date = i[:-2] + str(k)
  top_end_dates.append(end_date)
top_end_dates

['2015-07-21',
 '2015-07-29',
 '2015-07-30',
 '2015-07-31',
 '2015-08-18',
 '2015-08-19',
 '2015-08-20',
 '2015-08-21',
 '2015-09-9',
 '2015-09-10',
 '2016-07-7',
 '2016-07-8',
 '2016-07-16',
 '2016-07-19',
 '2016-07-23',
 '2016-07-24',
 '2016-07-26',
 '2016-07-27',
 '2016-07-28',
 '2016-07-29',
 '2016-08-11',
 '2016-08-12',
 '2016-08-13',
 '2016-08-14',
 '2016-08-15',
 '2016-08-16',
 '2016-08-17',
 '2016-08-18',
 '2016-09-10',
 '2017-06-13',
 '2017-06-14',
 '2017-06-24',
 '2017-07-12',
 '2017-07-13',
 '2017-07-14',
 '2017-07-18',
 '2017-07-19',
 '2017-07-20',
 '2017-07-21',
 '2017-07-22',
 '2017-07-23',
 '2017-08-2',
 '2017-08-3',
 '2017-08-4',
 '2017-08-5',
 '2017-08-19',
 '2017-08-23',
 '2017-09-28',
 '2018-07-2',
 '2018-07-3',
 '2018-07-4',
 '2018-07-6',
 '2018-07-17',
 '2018-07-18',
 '2018-08-3',
 '2018-08-7',
 '2018-08-8',
 '2018-08-9',
 '2018-08-10',
 '2018-08-17',
 '2018-08-18',
 '2018-08-29',
 '2018-08-30',
 '2018-08-31',
 '2018-09-5',
 '2018-09-6',
 '2018-09-7',
 '2019-07-4',

In [6]:
bottom_start_dates = bottom_anomaly_dates['Date'].to_list()
bottom_end_dates = []
for i in bottom_anomaly_dates.Date:
  k = int(i[8:]) + 1
  end_date = i[:-2] + str(k)
  bottom_end_dates.append(end_date)
bottom_end_dates

['2015-10-5',
 '2015-10-11',
 '2015-10-12',
 '2015-10-18',
 '2015-10-26',
 '2015-10-32',
 '2015-11-2',
 '2015-11-9',
 '2015-11-27',
 '2015-12-26',
 '2016-03-14',
 '2016-03-28',
 '2016-04-17',
 '2016-04-18',
 '2016-04-24',
 '2016-04-25',
 '2016-04-31',
 '2016-05-2',
 '2016-05-8',
 '2016-05-9',
 '2016-05-15',
 '2016-05-16',
 '2016-05-22',
 '2016-05-23',
 '2016-09-26',
 '2016-10-3',
 '2016-10-10',
 '2016-10-17',
 '2016-10-24',
 '2017-04-10',
 '2017-04-16',
 '2017-04-17',
 '2017-04-23',
 '2017-04-24',
 '2017-04-31',
 '2017-05-7',
 '2017-05-8',
 '2017-05-14',
 '2017-05-15',
 '2017-05-22',
 '2017-05-28',
 '2017-05-29',
 '2017-05-30',
 '2017-06-4',
 '2017-06-5',
 '2017-10-2',
 '2017-10-23',
 '2017-10-29',
 '2018-04-2',
 '2018-04-15',
 '2018-04-22',
 '2018-04-23',
 '2018-04-29',
 '2018-04-30',
 '2018-05-7',
 '2018-05-13',
 '2018-05-14',
 '2018-05-20',
 '2018-05-21',
 '2018-09-30',
 '2018-09-31',
 '2018-10-14',
 '2018-10-15',
 '2018-10-21',
 '2018-10-22',
 '2018-10-29',
 '2018-11-5',
 '2019-04-

In [11]:
def get_twitter_data(start_date_list: list, end_date_list: list, data_path: str, per_day=100):
    '''
    This function return a csv file and dataframe of all required twitter data for a keyword in the given date range.
    
    Inputs:
    keyword: Word for which we have to fetch twitter data
    start_date_list: List of starting dates 
    end_date_list: List of ending dates
    data_path: Path to store data
    per_day: Maximum number tweets required per day
    
    Output:
    Dataframe and CSV file of twitter data. 
    
    '''
    tweets_list = []
    tweet_df = pd.DataFrame()
    start_time = time.time()
    iteration_time = time.time()
    # Using TwitterSearchScraper to scrape data and append tweets to list
    for date_index in range(len(start_date_list)):
        for i,tweet in enumerate(sntwitter.TwitterSearchScraper('near: New York' + ' since:' + str(start_date_list[date_index])+' until:' +str(end_date_list[date_index])+' lang:en' + ' filter:has_engagement').get_items()):
            tweets_list.append([tweet.content, tweet.date, tweet.id])
            print(tweet.content)            
            tweet_df_temp = pd.DataFrame(tweets_list, columns=['text', 'date', 'Tweet Id'])
            tweet_df_temp.to_csv(data_path  + '_twitter_data.csv')
            if i>per_day:
                break; 
        iteration_time_temp = time.time()
        print('Time Elapsed = ' + str(iteration_time_temp - iteration_time))
        iteration_time = iteration_time_temp
        print('Tweet for '+str(start_date_list[date_index])+' collected')
    return tweet_df    

In [12]:
path = '/content/sample_data/' + 'top_anomaly'
get_twitter_data(top_start_dates, top_end_dates, path )

Streaming output truncated to the last 5000 lines.
Heading up to Syracuse. Sometimes I forget just how massive New York State is - 5.5 hours on a train and you’re still nowhere near Canada.
Say it with me kids: calling a racist “racist” is NOWHERE NEAR the same level of bad as being racist yourself @nytimes https://t.co/WvEgMSkSBE
@emoturkeynugget it’s near new york n pennsylvania n other states tht i dont rly care about snjsjsjs
@AUkeibro Donald Trump is the son of an immigrant mother and an immigrant Grandfather. Mary Anne MacLeod (1912–2000) was born in Tong, a small village near Stornoway, in the Western Isles of Scotland. At age 17, she immigrated to the United States and started working as a maid in New York.
Summertime in New York is experiencing bugs falling from the heavens (or depths of hell?) and landing on or near you at any given moment.
I'm old enough to VIVIDLY remember NYC in the 1970s and I'm still surprised by what I see every day near my office on West 35th Street - 

""


In [13]:
path = '/content/sample_data/' + 'bottom_anomaly'
get_twitter_data( bottom_start_dates, bottom_end_dates, path )

Streaming output truncated to the last 5000 lines.
@TerriGowin @AliciaHollinger @NewJeffCT @tedlieu @realDonaldTrump @nytimes @POTUS 51%?  Whose poll are you citing because I haven't seen him near that number.  Maybe a fox news poll I could understand being that high.
Eugene de Salignac. A motorman operates a trolley cars near Williamsburg Bridge. New York City (1924) https://t.co/p038uMpdjB
Proud to share that #UniversalLove vinyl album from @MGMResortsIntl and McCann New York is available today on #NationalRecordStoreDay. To find a copy near you:
https://t.co/Y7avmHX1j9. https://t.co/YwIKMpUrb1
"An extraordinarily tender documentary that asks what it means to be human ... even the most gentle scenes raise mighty questions.” — @nytimes 
Bring #SummerInTheForest to a theater near you! Here's how: https://t.co/gvPEUTfI2i https://t.co/aToBA01YLX
New York Times critic's pick Grace Jones: Bloodlight and Bami is now playing in New York, Los Angeles, Boston &amp; more and coming soon to many

""
